<a href="https://colab.research.google.com/github/Goodomenn/rvc-colab/blob/main/rvc-webui-original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi
! nvcc -V
! free -h

Sun May 11 20:28:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os

# @markdown # Mount Google Drive
data_path = "/content/data"
mount_gdrive = True  # @param {type:"boolean"}
gdrive_data_path = "/content/drive/MyDrive/AI/rvc-webui"  # @param {type:"string"}

if mount_gdrive:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    os.makedirs(gdrive_data_path, exist_ok=True)
    ! ln -s {data_path} {gdrive_data_path}
else:
    os.makedirs(data_path, exist_ok=True)

In [ ]:
# @markdown # Clone repository

%cd /content/
repository_url = "https://github.com/liujing04/Retrieval-based-Voice-Conversion-WebUI.git"  # @param {type: "string"}
webui_branch = "main"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/Retrieval-based-Voice-Conversion-WebUI
! git checkout {webui_branch}

! curl -LO https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt

pretrained = ["D32k.pth", "D40k.pth", "D48k.pth", "G32k.pth", "G40k.pth", "G48k.pth", "f0D32k.pth", "f0D40k.pth", "f0D48k.pth", "f0G32k.pth", "f0G40k.pth", "f0G48k.pth"]
uvr5_weights = ["HP2-人声vocals%2B非人声instrumentals.pth", "HP5-主旋律人声vocals%2B其他instrumentals.pth"]

for file in pretrained:
    ! cd pretrained && curl -OL https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/{file}
for file in uvr5_weights:
    ! cd uvr5_weights && curl -OL https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/{file}

In [ ]:
# @markdown # Initialize environment
import os

conda_dir = "/opt/conda"  # @param{type:"string"}
conda_bin = os.path.join(conda_dir, "bin", "conda")

if not os.path.exists(conda_bin):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p {conda_dir}
    ! rm Miniconda3-latest-Linux-x86_64.sh

def run_script(s):
    ! {s}

def make_args(d):
    arguments = ""
    for k, v in d.items():
        if type(v) == bool:
            arguments += f"--{k} " if v else ""
        elif type(v) == str and v:
            arguments += f"--{k} \"{v}\" "
        elif v:
            arguments += f"--{k}={v} "
    return arguments

if os.path.exists("requirments.txt"):
    ! mv requirments.txt requirements.txt

run_script(f"""
eval "$({conda_bin} shell.bash hook)"
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip install -r requirements.txt
pip uninstall faiss-cpu numpy -y
conda install -c conda-forge faiss numpy==1.23.5 -y
""")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  148M  100  148M    0     0   170M      0 --:--:-- --:--:-- --:--:--  170M
PREFIX=/opt/conda
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
installation finished.
    

In [ ]:
# @markdown # Launch webui
run_script(f"""
eval "$({conda_bin} shell.bash hook)"
python infer-web.py --colab
""")